# OpenReviewPy Tutorial


This tutorial will demonstrate how to use the OpenReviewPy library to manage your conference.




### Getting Started

Before you begin, make sure that you've installed the latest version of OpenReviewPy using pip:

```
pip install --upgrade openreviewpy
```


In [ ]:
# Start by importing all four OpenReviewPy classes
import openreview

### OpenReviewPy classes
OpenReviewPy includes four classes:
- **Client**: used to interact with the OpenReview servers.
- **Group**: represents an OpenReview *entity* 
    - E.g. organizations, conferences, individuals, reviewers, program committees, etc.
- **Note**: represents a *record* within OpenReview 
    - E.g. paper submissions, comments, reviews, etc.
- **Invitation**: represents an *invitation for users to submit notes*
    - E.g. a call for paper submissions, an invitation open to the public to submit comments, an invitation to only the program committee to submit a review, etc.

In [ ]:
# Let's start by initializing the Client with our username, password, and base URL
your_username = None #fill in your email address that you use to log in to OpenReview
your_password = None #fill in your password
your_baseurl  = None #fill in your desired baseurl (e.g. 'http://localhost:3000', or 'http://dev.openreview.net', etc.)

openreview_client = openreview.Client(username=your_username, password=your_password, baseurl=your_baseurl)

In [ ]:
# Get the list of accepted reviewers from the conference ICLR.cc/2017/conference
reviewers = openreview_client.get_group('ICLR.cc/2017/conference/reviewers')
print reviewers

In [ ]:
# Try to get the total list of invited reviewers
reviewers_invited = openreview_client.get_group('ICLR.cc/2017/conference/reviewers-invited')

# ...but fail because you don't have permission!
print reviewers_invited

### Posting a Note to DBLP
In this section we'll examine the invitation, "DBLP.org/-/paper", which have already been created for you. We'll also create an empty note, fill that note with the correct information, and then finally post a note to DBLP.org's invitation.

In [ ]:
DBLP_paper = openreview_client.get_invitation('DBLP.org/-/paper')

##second, create an empty Note object. We'll add information to this object as we go.
note = openreview.Note()

All invitations have a field called 'reply' which provides the specifications that a note must adhere to in order to be submitted.

The 'reply' field itself has a field called 'content', which describes the pieces of information that a note must have in order to be posted to the invitation. 

Let's see what's inside the reply.content field of 'DBLP.org/-/paper':

In [ ]:
DBLP_paper.reply['content']

The top level objects in the content field are the names of the contents that must be present in your note's content field. Each of these objects has a couple fields of their own. 'order' and 'required' specify details about how the UI is rendered; you can safely ignore these fields.

'value-regex', however, is important: this field contains the regex string that constrains the note's contents for that field. For example, we can see that the 'value-regex' field for 'title' is simply '.{0,100}'. This means that the title can consist of any character, repeated 0 to 100 times. (There are online regex tools that you can use to test your inputs - I like to use https://regex101.com/. The constraints for this particular invitation are very loose, so you shouldn't run into problems, but this can be very helpful for understanding bugs if they arise). 

There are other field types that are similar to 'value-regex' but specify different restrictions. For example, later we'll see the 'values' field, which constrains the field to a set of specific values.

Now that we know which pieces of information must be in the note's contents to be accepted by the invitation "DBLP.org/-/paper", let's add those fields to our note's content field, and see what the updated note looks like:

In [ ]:
# change these variables as you like

note.content = {'abstract': 'this is the abstract of a paper, submitted via OpenReviewPy',
 'authorids': 'spector@cs.umass.edu,michaelspector@gmail.com',
 'authors': 'Michael Spector', ## these are the authors exactly as they appear in the DBLP BibTeX record
 'DBLP_url': 'http://dblp.org/rec/conf/someconf/ABCxyz',
 'title': 'Michael\'s Paper'}

We're not ready to post the note just yet. We still need to add four more important pieces of information:
- the note's invitation: this is simply the invitation to which this note is posted ("DBLP.org/-/paper")
- the "signature" of the note: a list [ ] of groups that represent the user (e.g. "~Michael_Spector1")
- the note's "readers": a list [ ] of groups that have permission to read the note
- the note's "writers": a list [ ] of groups that have permission to modify the note

In a very similar way to our invitation's reply.contents, our invitation also specifies constraints on the signature, readers, and writers. Let's check them out:


In [ ]:
print 'signatures: ',DBLP_paper.reply['signatures']
print 'writers: ',DBLP_paper.reply['writers']
print 'readers: ',DBLP_paper.reply['readers']

Very much like each of our fields in reply.contents, the signatures, writers, and readers fields all have constraints on what values are allowed. But unlike the previous fields, here we're restricted to a set of specific values (in fact, just a single value for each): the signatures and writers must contain the group 'DBLP.org/upload', and the readers field must contain the group 'everyone'

(note: please also add the creation date, or 'cdate', of the paper. I've been told that DBLP entries include this data, but I'm not positive)

In [ ]:
note.invitation = 'DBLP.org/-/paper'
note.signatures = ['DBLP.org/upload'] #must be 'DBLP.org/upload', according to the invitation
note.writers = note.signatures #Should be the same as note.signatures
note.readers = ['everyone'] #must be 'everyone', according to the invitation
note.cdate=1234 #this is the date that the paper was created in milliseconds. DBLP should have this information per paper

Notice that you're about to sign a note as the group "DBLP.org/upload". This will only succeed if you have permission to sign a note with this signature. Let's check this by seeing if you are a signatory of the 'DBLP.org/upload' group:

In [ ]:
openreview_client.get_group('DBLP.org/upload').to_json()

You should see above that the "signatories", or the groups that are allowed to sign as "DBLP.org/upload", is the "DBLP.org/upload" group itself, which means that any member of "DBLP.org/upload" is allowed to sign notes using its signature.

Is your email address in the "members" field? If so, we're ready to post the note! Let's see what it looks like now.

In [ ]:
note.to_json()

You may notice that many of the fields are missing. This is OK - a lot of this information gets populated automatically by the database.

In [ ]:
openreview_client.post_note(note)

See if your note showed up by visiting <baseurl>/notes?invitation=DBLP.org/-/paper

In [ ]:
notes = openreview_client.get_notes(invitation='DBLP.org/-/paper')
for n in notes:
    print n.to_json()